<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-UT/blob/main/DataAlgo_UT(015)_TwoApprox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8. 近似アルゴリズム

多項式時間では解が求められない問題について、多項式時間で近似解を求めることを考える．  
近似解が厳密解にどれだけ近いか精度保証が出来る場合、近似解を与えるアルゴリズムを近似アルゴリズムと呼ぶ．



**いつもの約束**  
１つのコードセルだけの実行は Ctrl + Enter．  
エディタで「インデント幅（スペース）は4で表示」「行番号を表示」「インデントガイドを表示」．  
内部では日本語はUTF-8で表現されている．


# 準備

インスタンスに接続し起動する．  
下記の手順でGoogle Driveをマウントする．  
マウント先に移動し，作業フォルダとする．  
これによって，インスタンスがリセットされてもGoogle Drive内にファイルが保存されるようにする．

In [ ]:
!echo "Google Driveをマウントします"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "今回の作業用フォルダを作成しそこに移動します"
%cd /content/drive/My\ Drive/
%mkdir -p 202004_DataAlgo/DA2021_015
%cd       202004_DataAlgo/DA2021_015
!ls
!echo "日本時間表示"
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime
!date

# 8.1.２近似アルゴリズム

本節では，幾何制約のついた巡回セールスマン問題において、２近似アルゴリズムが得られることを学習する．  
このとき，多項式時間で近似解は得られるし，その解が示す経路値は最小経路値の２倍以内であることは制度保証されるが，最小経路値については不明のままであることに注意する．

# 巡回セールスマン問題 (Traveling Salesman's Problem / TSP)

**問題**

正の重み付きエッジによる無向連結グラフにおいて，全頂点を１度ずつ通る最短経路を求める．  
このとき，エッジの重みに三角不等式が成立するとき，幾何巡回セールス問題という．  
幾何がついてもつかなくても，巡回セールスマン問題は多項式時間では解けないとされている．  






# 全解探索法

全ての頂点の並び順を考えれば，それが解候補となる．  
経路として構成し得ることを確認し，その経路値を求める．  
全ての解候補の中で最小の経路値を示した解候補が最適解
である．  
頂点数をNとすると解候補は N! 個存在するため，多項式時間で求めることはできない．

# ２近似アルゴリズム

TSPのような最小化問題において，最適解の値のα倍以内の近似解を求めるアルゴリズムを，α近似アルゴリズムという．  

**アルゴリズム**

N頂点のグラフに対する２近似アルゴリズムを以下に示す．

1. 全頂点を含む最小全域木(Minimum Spanning Tree) T を求める．（多項式時間で得られる）
2. Tの全頂点を通る経路 C を求める．（ある頂点を根として深さ優先探索すれば求められる）
3. 経路 C において，すでに通過済みの頂点は訪れずに飛ばす新たな経路 C' を得る．

最小全域木とは，全頂点を含む木の中でその重みが最も小さいものをいう．  
C' が２近似解である．  

**計算時間**

手順1は後に占めるようにNについて多項式時間で得られる．
手順2,3は明らかにNについて多項式時間で実行できる．  
よって，近似解は多項式時間で求められる．

**精度**

今，グラフGのエッジの重み合計をw(G)で表すことにすると，明らかに下記が成立する．
このとき，Cでは同じエッジを必ず２回（行きと帰りに１回ずつ）通ることになることに注意する．

> w(C') <= w(C) = w(2T) = 2 w(T)

最適解を Co とすると，Co に含まれる辺はN本である．  
Tは最小全域木なので，必ず次が成立する．  
（そもそもTの方がエッジ数が１本少ないし，１本道（これも木の一種）より木の方が軽いものが構成できる）  

> w(T) < w(Co)

また，定義から次は自明．

> w(Co) < w(C')

これらから，次式を得る．

> w(C') / w(Co) < 2

このことは，近似解 C' の重みが最適解 Co の重みの2倍を常に超えないことを保証している．





# Primのアルゴリズム

最小全域木を多項式時間で求めるアルゴリズムの例として，PrimのアルゴリズムのCプログラム実装例を以下に示す．  
アルゴリズムについてはwikipedia等に解説がある．

https://ja.wikipedia.org/wiki/プリム法

**解説**

プログラムを自分で読んでみること．  
実はこれもまた，Greedy（でありながら最適解を得る）アルゴリズムである．


In [ ]:
%%writefile MST-Prim.c
// Prim's algorithm for MST by Neeraj Mishra
// (kameda[at]ccs.tsukuba.ac.jp, 2021.)
// https://www.thecrazyprogrammer.com/2014/06/prims-algorithm-and-program-for-minimum-cost-spanning-tree.html
#include<stdio.h>
#include<stdlib.h>
 
#define infinity 9999
#define MAX 20
 
int n; // number of edges in the graph
int edge[MAX][MAX];
int spanning[MAX][MAX];
 
int prims(void) {
    int cost[MAX][MAX];
	int u, v, min_distance, distance[MAX], from[MAX];
	int visited[MAX], no_of_edges, i, min_cost, j;
	
	//create cost[][] matrix,spanning[][]
	for (i = 0; i < n; i++)
		for (j = 0; j < n; j++) {
			if (edge[i][j] == 0)
				cost[i][j] = infinity;
			else 
				cost[i][j] = edge[i][j];
			spanning[i][j]=0;
            
		}
		
	//initialize visited[], distance[] and from[]
	distance[0] = 0;
	visited[0]  = 1;	
	for (i = 1; i < n; i++) {
		distance[i] = cost[0][i];
		from[i]     = 0;
		visited[i]  =0;
	}
	
	min_cost = 0; //cost of spanning tree
	no_of_edges = n-1; //no. of edges to be added
	
    // Core loop: add en edge one by one ...
	while (no_of_edges > 0) {
		//find the vertex at minimum distance from the tree ... Greedy!
		min_distance = infinity;
		for (i = 1; i < n; i++)
			if ( visited[i] == 0 && distance[i] < min_distance) {
				v = i;
				min_distance = distance[i];
			}
			
		u = from[v];
		
		//insert the edge in spanning tree
		spanning[u][v] = distance[v];
		spanning[v][u] = distance[v];
		no_of_edges--;
		visited[v]=1;
		
		//update the distance[] array
		for (i = 1; i < n; i++)
			if (visited[i] == 0 && cost[i][v] < distance[i]) {
				distance[i] = cost[i][v];
				from[i] = v;
			}
		
		min_cost = min_cost + cost[u][v];
	}
	
	return min_cost;
}

// Main function 
int main() {
	int i,j,total_cost;
	printf("Enter no. of vertices:");
	scanf("%d",&n);
	
	printf("\nEnter the adjacency matrix:\n");
	
	for(i = 0; i < n; i++)
		for(j = 0; j < n; j++)
			scanf("%d", &edge[i][j]);
	
	total_cost = prims();
	printf("\nspanning tree matrix:\n");
	
	for(i = 0; i < n; i++)
	{
		printf("\n");
		for(j = 0; j < n; j++)
			printf("%d\t", spanning[i][j]);
	}
	
	printf("\n\nTotal cost of spanning tree=%d",total_cost);
	return 0;
}
 

/****

Enter no. of vertices:6

Enter the adjacency matrix:
0 3 1 6 0 0
3 0 5 0 3 0
1 5 0 5 6 4
6 0 5 0 0 2
0 3 6 0 0 6
0 0 4 2 6 0

spanning tree matrix:

0 3 1 0 0 0
3 0 0 0 3 0
1 0 0 0 0 4
0 0 0 0 0 2
0 3 0 0 0 0
0 0 4 2 0 0

Total cost of spanning tree=13

****/


In [ ]:
!gcc -Wall -o MST-Prim MST-Prim.c

実行してみよう．最初に頂点数を，次に隣接行列を入力する．  
隣接行列は複数行纏めて渡すことができる．  

In [ ]:
!./MST-Prim

時間計算量が頂点数に対して多項式時間であることを確認しておくこと．．

# 節末課題

1. 計算量についての考察  
MST-Prim.c プログラムの時間計算量と空間計算量について考察せよ．その根拠を説明すること．  


2. 精度保証の導出
「これらから，次式を得る．」とある部分をじっさいに導出せよ．  







# 出典

筑波大学工学システム学類  
データ構造とアルゴリズム  
担当：亀田能成  
2021/06/23 初版